In [1]:
import os
import pickle
# Gmail API utils
from googleapiclient.discovery import build
from google_auth_oauthlib.flow import InstalledAppFlow
from google.auth.transport.requests import Request
# for encoding/decoding messages in base64
from base64 import urlsafe_b64decode, urlsafe_b64encode
# for dealing with attachement MIME types
from email.mime.text import MIMEText
from email.mime.multipart import MIMEMultipart
from email.mime.image import MIMEImage
from email.mime.audio import MIMEAudio
from email.mime.base import MIMEBase
from mimetypes import guess_type as guess_mime_type

# Request all access (permission to read/send/receive emails, manage the inbox, and more)
SCOPES = ['https://mail.google.com/']
our_email = 'theendernaut@gmail.com'

# All the below code was taken from the following link
https://www.thepythoncode.com/article/use-gmail-api-in-python


In [2]:
import os
import pickle
# Gmail API utils
from googleapiclient.discovery import build
from google_auth_oauthlib.flow import InstalledAppFlow
from google.auth.transport.requests import Request
# for encoding/decoding messages in base64
from base64 import urlsafe_b64decode, urlsafe_b64encode
# for dealing with attachement MIME types
from email.mime.text import MIMEText
from email.mime.multipart import MIMEMultipart
from email.mime.image import MIMEImage
from email.mime.audio import MIMEAudio
from email.mime.base import MIMEBase
from mimetypes import guess_type as guess_mime_type

# Request all access (permission to read/send/receive emails, manage the inbox, and more)
SCOPES = ['https://mail.google.com/']
our_email = 'your_gmail@gmail.com'

# Obviously, you need to change our_email to your address, make sure you use the email you created the API auth with.

# First of all, let's make a function that loads the credentials.json, does the authentication with Gmail API and returns a service object that can be used later in all in our upcoming functions:

def gmail_authenticate():
    creds = None
    # the file token.pickle stores the user's access and refresh tokens, and is
    # created automatically when the authorization flow completes for the first time
    if os.path.exists("token.pickle"):
        with open("token.pickle", "rb") as token:
            creds = pickle.load(token)
    # if there are no (valid) credentials availablle, let the user log in.
    if not creds or not creds.valid:
        if creds and creds.expired and creds.refresh_token:
            creds.refresh(Request())
        else:
            flow = InstalledAppFlow.from_client_secrets_file('creds.json', SCOPES)
            creds = flow.run_local_server(port=0)
        # save the credentials for the next run
        with open("token.pickle", "wb") as token:
            pickle.dump(creds, token)
    return build('gmail', 'v1', credentials=creds)

# get the Gmail API service
service = gmail_authenticate()

In [3]:
def search_messages(service, query):
    result = service.users().messages().list(userId='me',q=query).execute()
    messages = [ ]
    if 'messages' in result:
        messages.extend(result['messages'])
    while 'nextPageToken' in result:
        page_token = result['nextPageToken']
        result = service.users().messages().list(userId='me',q=query, pageToken=page_token).execute()
        if 'messages' in result:
            messages.extend(result['messages'])
    return messages


In [4]:
# utility functions
def get_size_format(b, factor=1024, suffix="B"):
    """
    Scale bytes to its proper byte format
    e.g:
        1253656 => '1.20MB'
        1253656678 => '1.17GB'
    """
    for unit in ["", "K", "M", "G", "T", "P", "E", "Z"]:
        if b < factor:
            return f"{b:.2f}{unit}{suffix}"
        b /= factor
    return f"{b:.2f}Y{suffix}"


def clean(text):
    # clean text for creating a folder
    return "".join(c if c.isalnum() else "_" for c in text)

In [5]:
def parse_parts(service, parts, message):
    """
    Utility function that parses the content of an email partition
    """
    if parts:
        for part in parts:
            mimeType = part.get("mimeType")
            body = part.get("body")
            data = body.get("data")
            part_headers = part.get("headers")
            if part.get("parts"):
                # recursively call this function when we see that a part
                # has parts inside
                parse_parts(service, part.get("parts"), message)
            if mimeType == "text/plain":
                # if the email part is text plain
                if data:
                    text = urlsafe_b64decode(data).decode()
            elif mimeType == "text/html":
                # if the email part is an HTML content
                # save the HTML file and optionally open it in the browser
                html = (urlsafe_b64decode(data))
                return html
            else:
                # attachment other than a plain text or HTML
                for part_header in part_headers:
                    part_header_name = part_header.get("name")
                    part_header_value = part_header.get("value")
                    if part_header_name == "Content-Disposition":
                        if "attachment" in part_header_value:
                            # we get the attachment ID 
                            # and make another request to get the attachment itself
                            attachment_id = body.get("attachmentId")
                            attachment = service.users().messages() \
                                        .attachments().get(id=attachment_id, userId='me', messageId=message['id']).execute()
                            data = attachment.get("data")
                            if data:
                                html = (urlsafe_b64decode(data))
                return html


In [6]:
def read_message(service, message):
    """
    This function takes Gmail API `service` and the given `message_id` and does the following:
        - Downloads the content of the email
        - Prints email basic information (To, From, Subject & Date) and plain/text parts
        - Creates a folder for each email based on the subject
        - Downloads text/html content (if available) and saves it under the folder created as index.html
        - Downloads any file that is attached to the email and saves it in the folder created
    """
    msg = service.users().messages().get(userId='me', id=message['id'], format='full').execute()
    # parts can be the message body, or attachments
    payload = msg['payload']
    
    parts = payload.get("parts")
        # since folders are created based on subjects
    data = parse_parts(service, parts, message)
    return data

In [7]:
pings = search_messages(service,'MIT')
html = read_message(service, pings[0])
# print(html)
from bs4 import BeautifulSoup as bs
soup = bs(html,'lxml')
# print(soup)
links = [a.get('href') for a in soup.find_all('a', href=True)]
images = [a.get('src') for a in soup.find_all('img')]
print(images)

['https://www.fastweb.com/nfs/fastweb/static/2019/notifications/upcoming-deadlines@2x_1200x270.gif', 'https://crcdn01.adnxs-simple.com/creative/p/3525/2021/11/30/30178320/212e9617-1c71-4201-af2d-92b1b7f5d803.jpg', 'https://lax1-ib.adnxs.com/ot?ps=2a0cffdc8487e159ba87cdd75ab06dcbfc6f936e&pe=wqT_3QJeTF4AAAADAOoABQEIgvS0jQYQAxkADQEAIQ0IbAAoxRswxRs4u6FHQISTzwdIBVACWIKqywFgAXkJIxTwP4IBCQkJDDAAAIgBAJABz-FZmAEA&pp=%24%7BAUCTION_PRICE%7D&an_audit=0&urlenc=https%3A%2F%2Fsecure.adnxs.com%2Fimptr%3Fid%3D48763%26cachebuster%3D1638742531687734470%26t%3D2', 'https://www.fastweb.com/nfs/fastweb/static/2017/notifications/facebook.png', 'https://www.fastweb.com/nfs/fastweb/static/2017/notifications/twitter.png', 'https://www.fastweb.com/nfs/fastweb/static/2017/notifications/pinterest.png', 'https://www.fastweb.com/nfs/fastweb/static/2017/notifications/instagram.png', 'https://www.fastweb.com/nfs/fastweb/static/2019/notifications/a-monster-company_250x24@2x.png', 'https://www.fastweb.com/uploads/widget_